In [529]:
import pandas as pd

In [530]:
df = pd.read_csv("../data/data_transformed.csv")

In [531]:
df.isna().sum()

event-id                        0
location-long                   0
location-lat                    0
argos:lat1                      0
argos:lat2                     85
argos:lon1                      0
argos:lon2                     85
individual-local-identifier     0
date                            0
time                            0
dtype: int64

In [532]:
df = df.dropna()

In [533]:
import datetime as dt

In [534]:
df["year"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").year)


In [535]:
df["month"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").month)

In [536]:
df=df.drop(columns=["event-id", "date"])

In [537]:
for elem in df["individual-local-identifier"].unique():
     df.loc[df["individual-local-identifier"]==elem].to_csv(f"../data/{elem}.csv")

In [538]:
df_23043 = pd.read_csv("./1999CA-Bmu-23043.csv") #individuo con la ruta más larga

In [539]:
df_23043

,Unnamed: 0,location-long,location-lat,argos:lat1,argos:lat2,argos:lon1,argos:lon2,individual-local-identifier,time,year,month
0,3173,-120.143,34.120,34.120,35.544,-120.143,-127.027,1999CA-Bmu-23043,16:09:13,1999,7
1,3174,-120.145,34.147,34.147,32.243,-120.145,-129.529,1999CA-Bmu-23043,03:27:53,1999,7
2,3175,-119.740,33.980,33.980,38.156,-119.740,-139.994,1999CA-Bmu-23043,12:33:13,1999,7
3,3176,-119.771,34.163,34.163,28.573,-119.771,-145.945,1999CA-Bmu-23043,23:53:08,1999,7
4,3177,-119.688,34.350,34.350,42.993,-119.688,-76.494,1999CA-Bmu-23043,03:20:07,1999,7
...,...,...,...,...,...,...,...,...,...,...,...
410,3583,-122.007,21.730,21.730,22.057,-122.007,-123.562,1999CA-Bmu-23043,16:00:28,1999,12
411,3584,-122.871,21.247,21.247,20.439,-122.871,-119.302,1999CA-Bmu-23043,12:00:55,1999,12
412,3585,-122.755,21.083,21.083,16.252,-122.755,-101.167,1999CA-Bmu-23043,17:16:22,1999,12
413,3586,-122.272,20.831,20.831,14.861,-122.272,-149.760,1999CA-Bmu-23043,04:21:51,1999,12


In [540]:
import tensorflow as tf
from tensorflow.data import Dataset

In [541]:
df_23043 = df_23043.drop(columns=["Unnamed: 0", "individual-local-identifier", "time", "year", "month"])
#ballena con ruta más larga

In [542]:
df_23043

,location-long,location-lat,argos:lat1,argos:lat2,argos:lon1,argos:lon2
0,-120.143,34.120,34.120,35.544,-120.143,-127.027
1,-120.145,34.147,34.147,32.243,-120.145,-129.529
2,-119.740,33.980,33.980,38.156,-119.740,-139.994
3,-119.771,34.163,34.163,28.573,-119.771,-145.945
4,-119.688,34.350,34.350,42.993,-119.688,-76.494
...,...,...,...,...,...,...
410,-122.007,21.730,21.730,22.057,-122.007,-123.562
411,-122.871,21.247,21.247,20.439,-122.871,-119.302
412,-122.755,21.083,21.083,16.252,-122.755,-101.167
413,-122.272,20.831,20.831,14.861,-122.272,-149.760


In [543]:
df_23043.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location-long  415 non-null    float64
 1   location-lat   415 non-null    float64
 2   argos:lat1     415 non-null    float64
 3   argos:lat2     415 non-null    float64
 4   argos:lon1     415 non-null    float64
 5   argos:lon2     415 non-null    float64
dtypes: float64(6)
memory usage: 19.6 KB


In [544]:
n = len(df_23043)
train_df = df_23043[0:int(n*0.7)]
val_df = df_23043[int(n*0.7):int(n*0.9)]
test_df = df_23043[int(n*0.9):]

num_features = df_23043.shape[1]


In [545]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
train_df = tf.keras.utils.timeseries_dataset_from_array(
    train_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [546]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
test_df = tf.keras.utils.timeseries_dataset_from_array(
    test_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [547]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
val_df = tf.keras.utils.timeseries_dataset_from_array(
    val_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [548]:
from tensorflow.data import Dataset

In [549]:
def mapdataset(elem):
    X,y = elem[:,:-1,:],elem[:,-1:,0:2]
    X.set_shape([None, n_points, None])
    y.set_shape([None, window_length - n_points, None])
    
    return X,y

In [550]:
train = train_df.map(mapdataset)

In [551]:
test = test_df.map(mapdataset)

In [552]:
val = val_df.map(mapdataset)

In [553]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=2)
])

In [554]:
MAX_EPOCHS = 300

def compile_and_fit(lstm_model, new_dataset, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                    patience=patience,
                                                    mode='min')

    lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

    history = lstm_model.fit(new_dataset, epochs=MAX_EPOCHS,
                      validation_data=val,
                      callbacks=[early_stopping])
    return history

In [555]:
history = compile_and_fit(lstm_model, train)

Epoch 1/300
9/9 [==============================] - 2s 52ms/step - loss: 7738.7627 - mean_absolute_error: 77.1633 - val_loss: 7435.9487 - val_mean_absolute_error: 75.1138
Epoch 2/300
9/9 [==============================] - 0s 17ms/step - loss: 7670.0610 - mean_absolute_error: 76.7601 - val_loss: 7373.3218 - val_mean_absolute_error: 74.7464
Epoch 3/300
9/9 [==============================] - 0s 17ms/step - loss: 7601.9951 - mean_absolute_error: 76.3871 - val_loss: 7286.3921 - val_mean_absolute_error: 74.3355
Epoch 4/300
9/9 [==============================] - 0s 17ms/step - loss: 7505.1025 - mean_absolute_error: 75.9791 - val_loss: 7202.1953 - val_mean_absolute_error: 73.9766
Epoch 5/300
9/9 [==============================] - 0s 17ms/step - loss: 7435.4243 - mean_absolute_error: 75.6557 - val_loss: 7151.4707 - val_mean_absolute_error: 73.7212
Epoch 6/300
9/9 [==============================] - 0s 21ms/step - loss: 7383.9307 - mean_absolute_error: 75.3903 - val_loss: 7109.9248 - val_mean_abso

Epoch 49/300
9/9 [==============================] - 0s 16ms/step - loss: 6249.1919 - mean_absolute_error: 67.6187 - val_loss: 6011.6128 - val_mean_absolute_error: 65.7528
Epoch 50/300
9/9 [==============================] - 0s 17ms/step - loss: 6226.8838 - mean_absolute_error: 67.4578 - val_loss: 5989.9287 - val_mean_absolute_error: 65.5922
Epoch 51/300
9/9 [==============================] - 0s 16ms/step - loss: 6204.6694 - mean_absolute_error: 67.2973 - val_loss: 5968.3364 - val_mean_absolute_error: 65.4319
Epoch 52/300
9/9 [==============================] - 0s 17ms/step - loss: 6182.5479 - mean_absolute_error: 67.1373 - val_loss: 5946.8340 - val_mean_absolute_error: 65.2721
Epoch 53/300
9/9 [==============================] - 0s 17ms/step - loss: 6160.5151 - mean_absolute_error: 66.9776 - val_loss: 5925.4189 - val_mean_absolute_error: 65.1126
Epoch 54/300
9/9 [==============================] - 0s 17ms/step - loss: 6138.5742 - mean_absolute_error: 66.8182 - val_loss: 5904.0933 - val_mea

Epoch 97/300
9/9 [==============================] - 0s 21ms/step - loss: 5269.1528 - mean_absolute_error: 60.2808 - val_loss: 5059.6548 - val_mean_absolute_error: 58.4241
Epoch 98/300
9/9 [==============================] - 0s 18ms/step - loss: 5250.4814 - mean_absolute_error: 60.1355 - val_loss: 5041.5356 - val_mean_absolute_error: 58.2789
Epoch 99/300
9/9 [==============================] - 0s 16ms/step - loss: 5231.8745 - mean_absolute_error: 59.9904 - val_loss: 5023.4785 - val_mean_absolute_error: 58.1340
Epoch 100/300
9/9 [==============================] - 0s 17ms/step - loss: 5213.3320 - mean_absolute_error: 59.8456 - val_loss: 5005.4854 - val_mean_absolute_error: 57.9894
Epoch 101/300
9/9 [==============================] - 0s 17ms/step - loss: 5194.8525 - mean_absolute_error: 59.7012 - val_loss: 4987.5537 - val_mean_absolute_error: 57.8451
Epoch 102/300
9/9 [==============================] - 0s 16ms/step - loss: 5176.4355 - mean_absolute_error: 59.5570 - val_loss: 4969.6846 - val_

Epoch 145/300
9/9 [==============================] - 0s 16ms/step - loss: 4370.6558 - mean_absolute_error: 53.0082 - val_loss: 4187.8237 - val_mean_absolute_error: 51.1527
Epoch 146/300
9/9 [==============================] - 0s 16ms/step - loss: 4353.5806 - mean_absolute_error: 52.8651 - val_loss: 4171.3081 - val_mean_absolute_error: 51.0100
Epoch 147/300
9/9 [==============================] - 0s 16ms/step - loss: 4336.5967 - mean_absolute_error: 52.7227 - val_loss: 4154.8789 - val_mean_absolute_error: 50.8679
Epoch 148/300
9/9 [==============================] - 0s 16ms/step - loss: 4319.7007 - mean_absolute_error: 52.5808 - val_loss: 4138.5332 - val_mean_absolute_error: 50.7264
Epoch 149/300
9/9 [==============================] - 0s 17ms/step - loss: 4302.8906 - mean_absolute_error: 52.4395 - val_loss: 4122.2700 - val_mean_absolute_error: 50.5854
Epoch 150/300
9/9 [==============================] - 0s 17ms/step - loss: 4286.1631 - mean_absolute_error: 52.2988 - val_loss: 4106.0869 - v

9/9 [==============================] - 0s 17ms/step - loss: 3643.1228 - mean_absolute_error: 46.7855 - val_loss: 3483.9890 - val_mean_absolute_error: 44.9417
Epoch 193/300
9/9 [==============================] - 0s 16ms/step - loss: 3629.0125 - mean_absolute_error: 46.6623 - val_loss: 3470.3428 - val_mean_absolute_error: 44.8188
Epoch 194/300
9/9 [==============================] - 0s 17ms/step - loss: 3614.9514 - mean_absolute_error: 46.5395 - val_loss: 3456.7437 - val_mean_absolute_error: 44.6973
Epoch 195/300
9/9 [==============================] - 0s 17ms/step - loss: 3600.9397 - mean_absolute_error: 46.4171 - val_loss: 3443.1914 - val_mean_absolute_error: 44.5783
Epoch 196/300
9/9 [==============================] - 0s 17ms/step - loss: 3586.9756 - mean_absolute_error: 46.2950 - val_loss: 3429.6865 - val_mean_absolute_error: 44.4624
Epoch 197/300
9/9 [==============================] - 0s 17ms/step - loss: 3573.0603 - mean_absolute_error: 46.1733 - val_loss: 3416.2292 - val_mean_absolu

Epoch 240/300
9/9 [==============================] - 0s 38ms/step - loss: 3016.4963 - mean_absolute_error: 41.2490 - val_loss: 2878.0945 - val_mean_absolute_error: 39.9864
Epoch 241/300
9/9 [==============================] - 0s 37ms/step - loss: 3004.4487 - mean_absolute_error: 41.1415 - val_loss: 2866.4480 - val_mean_absolute_error: 39.8952
Epoch 242/300
9/9 [==============================] - 0s 26ms/step - loss: 2992.4392 - mean_absolute_error: 41.0342 - val_loss: 2854.8381 - val_mean_absolute_error: 39.8042
Epoch 243/300
9/9 [==============================] - 0s 23ms/step - loss: 2980.4666 - mean_absolute_error: 40.9273 - val_loss: 2843.2644 - val_mean_absolute_error: 39.7135
Epoch 244/300
9/9 [==============================] - 0s 18ms/step - loss: 2968.5322 - mean_absolute_error: 40.8207 - val_loss: 2831.7278 - val_mean_absolute_error: 39.6236
Epoch 245/300
9/9 [==============================] - 0s 19ms/step - loss: 2956.6343 - mean_absolute_error: 40.7144 - val_loss: 2820.2261 - v

9/9 [==============================] - 0s 18ms/step - loss: 2488.4634 - mean_absolute_error: 36.5135 - val_loss: 2367.6821 - val_mean_absolute_error: 36.0069
Epoch 288/300
9/9 [==============================] - 0s 26ms/step - loss: 2478.0237 - mean_absolute_error: 36.4195 - val_loss: 2357.5918 - val_mean_absolute_error: 35.9289
Epoch 289/300
9/9 [==============================] - 0s 20ms/step - loss: 2467.6152 - mean_absolute_error: 36.3258 - val_loss: 2347.5308 - val_mean_absolute_error: 35.8510
Epoch 290/300
9/9 [==============================] - 0s 17ms/step - loss: 2457.2375 - mean_absolute_error: 36.2324 - val_loss: 2337.5002 - val_mean_absolute_error: 35.7733
Epoch 291/300
9/9 [==============================] - 0s 16ms/step - loss: 2446.8906 - mean_absolute_error: 36.1392 - val_loss: 2327.5000 - val_mean_absolute_error: 35.6962
Epoch 292/300
9/9 [==============================] - 0s 16ms/step - loss: 2436.5747 - mean_absolute_error: 36.0462 - val_loss: 2317.5286 - val_mean_absolu

TypeError: 'MapDataset' object is not subscriptable